# Ensembling & Optimization
### Goal: Maximize performance
## Methods
 - Weighted averaging
 - Stacking (ML + DL)
 - Bias correction
 - Post-processing (zero-sales handling)
### 📌 Output: Final optimized model

In [ ]:
# Clone GitHub Repository
!git clone https://github.com/sabin74/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform.git

Cloning into 'Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 329 (delta 60), reused 37 (delta 8), pack-reused 216 (from 1)
Receiving objects: 100% (329/329), 41.98 MiB | 22.15 MiB/s, done.
Resolving deltas: 100% (172/172), done.
Updating files: 100% (58/58), done.
Filtering content: 100% (22/22), 348.58 MiB | 44.03 MiB/s, done.


In [ ]:
!pip install -q catboost
!pip install category_encoders


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.5 MB/s eta 0:00:00


In [ ]:

# Environment Setup - Import Libraries
import os
import gc
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Set Project Root
os.chdir("/content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform")
print("Current Directory: ", os.getcwd())

Current Directory:  /content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform


In [ ]:
# Load Feature-Engineered Data
DATA_DIR = Path("data/features")

train = pd.read_parquet(DATA_DIR / "train_features.parquet")
test = pd.read_parquet(DATA_DIR / "test_features.parquet")

In [ ]:
# Memory Optimization (reduce memory usage)
def reduce_mem_usage(df, ):
  for col in df.columns:
    if df[col].dtype == "float64":
      df[col] = df[col].astype("float32")
    elif df[col].dtype == "int64":
      df[col] = df[col].astype("int32")
  return df

train = reduce_mem_usage(train)
gc.collect()


0